In [69]:
from rucio.client import Client

#help(Client)

In [70]:
import os
Rucio_cert = os.getenv('X509_USER_CERT')
Rucio_key = os.getenv('X509_USER_KEY')
Rucio_host = 'https://cms-rucio.cern.ch'
Rucio_account = 'belforte'
Rucio_authUrl = 'https://cms-rucio-auth.cern.ch'
Rucio_caPath = '/etc/grid-security/certificates/'

nativeClient = Client(
    rucio_host=Rucio_host, auth_host=Rucio_authUrl, ca_cert=Rucio_caPath,
    account=Rucio_account,
    creds={"client_cert": Rucio_cert, "client_key": Rucio_key},
    auth_type = 'x509'
)
ret=nativeClient.ping()
print(ret['version'])
ret = nativeClient.whoami()
print(ret)

1.23.11.post3
{u'status': u'ACTIVE', u'account': u'belforte', u'account_type': u'USER', u'created_at': u'2020-04-28T23:25:20', u'updated_at': u'2020-04-28T23:25:20', u'suspended_at': None, u'deleted_at': None, u'email': u'Stefano.Belforte@cern.ch'}


In [71]:
import os
import sys
import pprint
import logging

from WMCore.Services.Rucio.Rucio import Rucio

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logger = logging.getLogger('Logger')


In [72]:

logger.info("Initializing WMCore Rucio client")
logger.setLevel(logging.ERROR)
rucio_config_dict = {
    "phedexCompatible": True,
    "auth_type": "x509", "ca_cert": '/etc/grid-security/certificates/',
    "logger": logger,
    "creds": {"client_cert": '/data/certs/servicecert.pem', "client_key": '/data/certs/servicekey.pem'}
}
user='belforte'
username = user
rucioClient = Rucio(
    user,
    hostUrl='https://cms-rucio.cern.ch',
    authUrl='https://cms-rucio-auth.cern.ch',
    configDict=rucio_config_dict
)
rucioClient.whoAmI()
logger.setLevel(logging.INFO)
logger.info("WMCore Rucio client OK")


INFO:Logger:Initializing WMCore Rucio client
INFO:Logger:WMCore Rucio client OK


In [73]:
# in CRAB tape requests start with a block list
# start with a dataset on tape
dataset = '/MuonEG/Run2016B-v1/RAW'

# can't use DBS Api in this Jupyter. Ask Rucio via WMCore wrapper
blocks = rucioClient.getBlocksInContainer(dataset)

logger.info('found %d blocks', len(blocks))
pprint.pprint(blocks)


INFO:Logger:found 4 blocks
[u'/MuonEG/Run2016B-v1/RAW#86bc5e3e-1519-11e6-a3f4-001e67ac06a0',
 u'/MuonEG/Run2016B-v1/RAW#898359ee-1760-11e6-8d9e-001e67ac06a0',
 u'/MuonEG/Run2016B-v1/RAW#9da47a68-1682-11e6-8d9e-001e67ac06a0',
 u'/MuonEG/Run2016B-v1/RAW#cd4b1b62-13aa-11e6-a3f4-001e67ac06a0']


In [74]:
dids = [{'scope':'cms','name':block} for block in blocks]
#dids = [{'scope':'user.belforte:','name':block} for block in blocks]
pprint.pprint(dids)

[{'name': u'/MuonEG/Run2016B-v1/RAW#86bc5e3e-1519-11e6-a3f4-001e67ac06a0',
  'scope': 'cms'},
 {'name': u'/MuonEG/Run2016B-v1/RAW#898359ee-1760-11e6-8d9e-001e67ac06a0',
  'scope': 'cms'},
 {'name': u'/MuonEG/Run2016B-v1/RAW#9da47a68-1682-11e6-8d9e-001e67ac06a0',
  'scope': 'cms'},
 {'name': u'/MuonEG/Run2016B-v1/RAW#cd4b1b62-13aa-11e6-a3f4-001e67ac06a0',
  'scope': 'cms'}]


In [75]:
#print(len(blocks))
size = 0
for block in blocks:
    block_replicas = nativeClient.list_dataset_replicas('cms',block)
    bytes = block_replicas.next()['bytes']
    print (bytes)
    size += bytes
#replicas=list(block_replicas)
print('total = %d GBytes' % (size / 1e9))
print('total = %d TBytes' % (size / 1e12))

16797810203
121278508448
2191775241
2240775706
total = 142 GBytes
total = 0 TBytes


In [77]:
#let's assume we want recall 2 blocks
didsToRecall=dids[0:2]
print(didsToRecall)
# make a new container to hold them, embedding task name
taskName='201120_131722:belforte_crab_20201120_141717'.replace(':','.')
print(taskName)
ctrName='/TapeRecall/%s/USER' % taskName
print(ctrName)
created = rucioClient.createContainer(ctrName, scope='user.belforte')
logger.info('contrainer creation is %s', created)

[{'scope': 'cms', 'name': u'/MuonEG/Run2016B-v1/RAW#86bc5e3e-1519-11e6-a3f4-001e67ac06a0'}, {'scope': 'cms', 'name': u'/MuonEG/Run2016B-v1/RAW#898359ee-1760-11e6-8d9e-001e67ac06a0'}]
201120_131722.belforte_crab_20201120_141717
/TapeRecall/201120_131722.belforte_crab_20201120_141717/USER
INFO:Logger:contrainer creation is True


In [78]:
containerName = ctrName
myScope = 'user.belforte'
containerDid = {'scope':myScope, 'name':containerName}

In [79]:
# add blocks (datasets in Rucio language) to container
pprint.pprint(didsToRecall)
pprint.pprint(containerName)
#ret=nativeCli.add_datasets_to_container('user.belforte', containerName, didsToRecall)
#ret=nativeCli.add_containers_to_container('user.belforte', containerName, didsToRecall)
ret = nativeClient.attach_dids('user.belforte', containerName, didsToRecall)

[{'name': u'/MuonEG/Run2016B-v1/RAW#86bc5e3e-1519-11e6-a3f4-001e67ac06a0',
  'scope': 'cms'},
 {'name': u'/MuonEG/Run2016B-v1/RAW#898359ee-1760-11e6-8d9e-001e67ac06a0',
  'scope': 'cms'}]
'/TapeRecall/201120_131722.belforte_crab_20201120_141717/USER'


DuplicateContent: Data identifier already added to the destination content.
Details: [u'(cx_Oracle.IntegrityError) ORA-00001: unique constraint (CMS_RUCIO_PROD.CONTENTS_PK) violated']

In [80]:
print(containerName)
print(ctrName)
cc=nativeClient.list_content('user.belforte',containerName)
content = [i for i in cc]
print(content)

/TapeRecall/201120_131722.belforte_crab_20201120_141717/USER
/TapeRecall/201120_131722.belforte_crab_20201120_141717/USER
[{u'adler32': None, u'name': u'/MuonEG/Run2016B-v1/RAW#86bc5e3e-1519-11e6-a3f4-001e67ac06a0', u'bytes': None, u'scope': u'cms', u'type': u'DATASET', u'md5': None}, {u'adler32': None, u'name': u'/MuonEG/Run2016B-v1/RAW#898359ee-1760-11e6-8d9e-001e67ac06a0', u'bytes': None, u'scope': u'cms', u'type': u'DATASET', u'md5': None}, {u'adler32': None, u'name': u'/MuonEG/Run2016B-v1/RAW#9da47a68-1682-11e6-8d9e-001e67ac06a0', u'bytes': None, u'scope': u'cms', u'type': u'DATASET', u'md5': None}]


In [81]:
did0=didsToRecall[0]
did1=didsToRecall[1]
did2=dids[2]
print(did0)
print(did1)
print(did2)


{'scope': 'cms', 'name': u'/MuonEG/Run2016B-v1/RAW#86bc5e3e-1519-11e6-a3f4-001e67ac06a0'}
{'scope': 'cms', 'name': u'/MuonEG/Run2016B-v1/RAW#898359ee-1760-11e6-8d9e-001e67ac06a0'}
{'scope': 'cms', 'name': u'/MuonEG/Run2016B-v1/RAW#9da47a68-1682-11e6-8d9e-001e67ac06a0'}


In [83]:
print(containerName)
print(did1)
# can't use WMCore/ServicesRucio.attachDIDs because it does not support dids and container to be in different scope
ret=nativeClient.attach_dids('user.belforte', containerName, [did2])

/TapeRecall/201120_131722.belforte_crab_20201120_141717/USER
{'scope': 'cms', 'name': u'/MuonEG/Run2016B-v1/RAW#898359ee-1760-11e6-8d9e-001e67ac06a0'}


DuplicateContent: Data identifier already added to the destination content.
Details: [u'(cx_Oracle.IntegrityError) ORA-00001: unique constraint (CMS_RUCIO_PROD.CONTENTS_PK) violated']

In [84]:
ret=nativeClient.list_content('user.belforte', containerName)
content = [c for c in ret]

In [86]:
#pprint.pprint(content)

In [91]:
RSE_EXPRESSION = 'ddm_quota>0&tier=2&rse_type=DISK'
RSE_EXPRESSION = 'T3_IT_Trieste'
rl=nativeClient.list_rses(RSE_EXPRESSION)
for r in rl:
    print(r)
WEIGHT = 'ddm_quota'
DAYS = 14 * 24 * 3600
copies = 1

{u'rse': u'T3_IT_Trieste'}


In [93]:
# createReplicationRule(self, names, rseExpression, scope='cms', copies=1, **kwargs):
rules = nativeClient.add_replication_rule([containerDid], copies, 'T3_IT_Trieste',
                                  weight=None, lifetime=DAYS, account=user, grouping='ALL',
                                  activity='Analysis Input', comment='Staged from tape for %s' % username,
                                  ask_approval=True, asynchronous=True,
                                           )

In [94]:
print(rules)

[u'5cc7dca7f29647c684d74461bdfeb3fa']


In [ ]:
resp = nativeClient.list_rule

In [ ]:
# createReplicationRule(self, names, rseExpression, scope='cms', copies=1, **kwargs):
rules = rucioClient.createReplicationRule(containerName, 'T3_IT_Trieste', scope='user.belforte', copies=1,
                                  weight=None, lifetime=DAYS, account=user,
                                  activity='Analysis Input', comment='Staged from tape for %s' % username,
                                  ask_approval=True, asynchronous=True,
                                           )

In [ ]:
rule = rules[0]

print("Rule %s has been created for %s" % (rule, user))
print("This rule can be monitored through DAS by checking the dataset or directly through Rucio with:")
print(" rucio rule-info %s" % rule)

rule_info = rucli.get_replication_rule(rule)

print('This rule expires at %s after which the data may be removed if not used occassionally'
      % rule_info['expires_at'])

print("This rule can be found by asking Rucio with:")
print(" rucio list-rules %s:%s" % ('user.belforte', containerName))

In [60]:
ruleId='cf9db5949a894efc9f39805a64ac275a'
resp=nativeClient.get_replication_rule(ruleId)

In [68]:
rule_info=resp
rule = ruleId


print("Rule %s has been created for user %s" % (rule, user))
print("This rule can be monitored through DAS by checking the dataset or directly through Rucio with:")
print(" rucio rule-info %s" % rule)

rule_info = nativeClient.get_replication_rule(rule)

print('This rule expires at %s after which the data may be removed if not used'
      % rule_info['expires_at'])

print("This rule can be found by asking Rucio with:")
print(" rucio list-rules %s:%s" % ('user.belforte', containerName))

Rule cf9db5949a894efc9f39805a64ac275a has been created for user belforte
This rule can be monitored through DAS by checking the dataset or directly through Rucio with:
 rucio rule-info cf9db5949a894efc9f39805a64ac275a
This rule expires at 2020-12-21 17:54:43 after which the data may be removed if not used
This rule can be found by asking Rucio with:
 rucio list-rules user.belforte:/TapeRecall/201120_131722.belforte_crab_20201120_141717/USER


In [61]:
pprint.pprint(resp)

{u'account': u'belforte',
 u'activity': u'Analysis Input',
 u'child_rule_id': None,
 u'comments': u'Staged from tape for belforte',
 u'copies': 1,
 u'created_at': datetime.datetime(2020, 12, 21, 16, 9, 1),
 u'did_type': u'CONTAINER',
 u'eol_at': None,
 u'error': None,
 u'expires_at': datetime.datetime(2020, 12, 21, 17, 54, 43),
 u'grouping': u'DATASET',
 u'id': u'cf9db5949a894efc9f39805a64ac275a',
 u'ignore_account_limit': True,
 u'ignore_availability': False,
 u'locked': False,
 u'locks_ok_cnt': 55,
 u'locks_replicating_cnt': 14,
 u'locks_stuck_cnt': 0,
 u'meta': None,
 u'name': u'/TapeRecall/201120_131722.belforte_crab_20201120_141717/USER',
 u'notification': u'NO',
 u'priority': 3,
 u'purge_replicas': False,
 u'rse_expression': u'T2_IT_Pisa',
 u'scope': u'user.belforte',
 u'source_replica_expression': None,
 u'split_container': False,
 u'state': u'REPLICATING',
 u'stuck_at': None,
 u'subscription_id': None,
 u'updated_at': datetime.datetime(2020, 12, 21, 17, 2, 53),
 u'weight': None